## 1.去掉取值变化小的特征 Removing features with low variance

再从接下来提到的的特征选择方法中选择合适的进行进一步的特征选择。

## 2.单变量特征选择 Univariate feature selection

### 2.1 Pearson相关系数 Pearson Correlation
皮尔森相关系数是一种最简单的，能帮助理解特征和响应变量之间关系的方法，该方法衡量的是变量之间的线性相关性，结果的取值区间为[-1，1]，-1表示完全的负相关(这个变量下降，那个就会上升)，+1表示完全的正相关，0表示没有线性相关。

Pearson Correlation速度快、易于计算，经常在拿到数据(经过清洗和特征提取之后的)之后第一时间就执行。Scipy的 pearsonr 方法能够同时计算相关系数和p-value，

In [1]:
import numpy as np
from scipy.stats import pearsonr

In [2]:
np.random.seed(0)
size = 300

In [3]:
x = np.random.normal(0, 1, size)
len(x)

300

1）输入：x为特征，y为目标变量.<br>
2）输出：r： 相关系数 [-1，1]之间，p-value: p值。<br>
     注： p值越小，表示相关系数越显著，一般p值在500个样本以上时有较高的可靠性。

In [4]:
print ("  Lower noise", pearsonr(x, x + np.random.normal(0, 1, size)))
print ("  Higher noise", pearsonr(x, x + np.random.normal(0, 10, size)))

  Lower noise (0.7182483686213841, 7.32401731299835e-49)
  Higher noise (0.057964292079338155, 0.3170099388532475)


# scikit-learn中树算法

In [11]:
from sklearn import metrics
from sklearn.ensemble import ExtraTreesClassifier
model = ExtraTreesClassifier()
model.fit(X, y)
# display the relative importance of each attribute
print(model.feature_importances_)

NameError: name 'X' is not defined

# RFE搜索算法 

基于对特征子集的高效搜索，从而找到最好的子集

In [ ]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
# create the RFE model and select 3 attributes
rfe = RFE(model, 3)
rfe = rfe.fit(X, y)
# summarize the selection of the attributes
print(rfe.support_)
print(rfe.ranking_)

# PCA+Pipeline

In [ ]:
fromsklearn.decomposition import PCA

steps =[
    ("pca", PCA()),
    ("LG", LogisticRegression())
]

pip =Pipeline(steps)

parameters={'pca__n_components':np.arange(25,49,3)}

model =GridSearchCV(pip, param_grid=parameters, scoring='accuracy',cv=10)


# 利用LassoCV进行特征选择

In [ ]:
import pandas as pd
import numpy as np
import csv as csv
import matplotlib
import matplotlib.pyplot as plt
from sklearn.linear_model import Ridge, RidgeCV, ElasticNet, LassoCV, LassoLarsCV
from sklearn.model_selection import cross_val_score



train = pd.read_csv('train.csv', header=0)        # Load the train file into a dataframe
df = pd.get_dummies(train.iloc[:,1:-1])
df = df.fillna(df.mean())

X_train = df
y = train.price



def rmse_cv(model):
    rmse= np.sqrt(-cross_val_score(model, X_train, y, scoring="neg_mean_squared_error", cv = 3))
    return(rmse)

#调用LassoCV函数，并进行交叉验证，默认cv=3
model_lasso = LassoCV(alphas = [0.1,1,0.001, 0.0005]).fit(X_train, y)

#模型所选择的最优正则化参数alpha
print(model_lasso.alpha_)

#各特征列的参数值或者说权重参数，为0代表该特征被模型剔除了
print(model_lasso.coef_)

#输出看模型最终选择了几个特征向量，剔除了几个特征向量
coef = pd.Series(model_lasso.coef_, index = X_train.columns)
print("Lasso picked " + str(sum(coef != 0)) + " variables and eliminated the other " +  str(sum(coef == 0)) + " variables")

#输出所选择的最优正则化参数情况下的残差平均值，因为是3折，所以看平均值
print(rmse_cv(model_lasso).mean())


#画出特征变量的重要程度，这里面选出前3个重要，后3个不重要的举例
imp_coef = pd.concat([coef.sort_values().head(3),
                     coef.sort_values().tail(3)])

matplotlib.rcParams['figure.figsize'] = (8.0, 10.0)
imp_coef.plot(kind = "barh")
plt.title("Coefficients in the Lasso Model")
plt.show() 

从上述代码中可以看出，权重为0的特征就是被剔除的特征，从而进行了特征选择。还可以从图上直观看出哪些特征最重要。至于权重为负数的特征，还需要进一步分析研究。

# 可视化特征选择

In [ ]:
# 散点图，如果有异常最大、最小值，可以进行极值的截断

plt.figure(figsize=(8,6))
plt.scatter(range(train.shape[0]), np.sort(train.price_doc.values))
plt.xlabel('index', fontsize=12)
plt.ylabel('price', fontsize=12)
plt.show()

#截断极值，因为极值有时候可以认为是异常数值，会干扰模型的参数
ulimit = np.percentile(train.price_doc.values, 99)
llimit = np.percentile(train.price_doc.values, 1)

train['price_doc'].ix[train['price_doc']>ulimit] = ulimit
train['price_doc'].ix[train['price_doc']<llimit] = llimit

## 柱图，分组进行分析

In [ ]:
grouped_df = df.groupby('LotFrontage')['MSSubClass'].aggregate(np.mean).reset_index() #根据LotFrontage进行分组聚合，并求出分组聚合后MSSubClass的平均值,reset_index()将分组后的结果转换成DataFrame形式

plt.figure(figsize=(12,8))
sns.barplot(grouped_df.LotFrontage.values, grouped_df.MSSubClass.values, alpha=0.9, color='red')
plt.ylabel('MSSubClass', fontsize=12)
plt.xlabel('LotFrontage', fontsize=12)
plt.xticks(rotation='vertical')
plt.show()